# Vectorizing tabular fields

This notebook covers simple methods to vectorize tabular data using the same dataset as in the other examples, but this time ignoring the text of the question.

In [1]:
import pandas as pd
from pathlib import Path
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings('ignore')

from ml_editor.data_processing import get_normalized_series

data_path = Path("../data/writers.csv")
df = pd.read_csv(data_path)

Let's pretend we wanted to predict the **score** from the tags, number of comments, and question creation date. Here is what the data looks like

In [2]:
df["is_question"] = df["PostTypeId"] == 1

tabular_df = df[df["is_question"]][["Tags", "CommentCount", "CreationDate", "Score"]]
tabular_df.head()

,Tags,CommentCount,CreationDate,Score
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,20
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,34
3,<plot><short-story><planning><brainstorming>,0,2010-11-18T20:43:59.693,28
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,21


In order to use this data as input to a model, we need to give it a suitable numerical representation. To do so, we will do three things here:

1. Normalize numerical input features to limit the impact of outliers

2. Transform the date feature in a way that makes it easier to understand for a model.

3. Get dummy variables from categorical features so a model can ingest them.

First, we normalize the data to reduce the effect of outliers on downstream model performance.

In [3]:
tabular_df["NormComment"]= get_normalized_series(tabular_df, "CommentCount")
tabular_df["NormScore"]= get_normalized_series(tabular_df, "Score")

In [4]:
tabular_df.head()

,Tags,CommentCount,CreationDate,Score,NormComment,NormScore
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32,1.405553,3.66092
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,20,-0.878113,2.02388
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,34,-0.551875,3.93376
3,<plot><short-story><planning><brainstorming>,0,2010-11-18T20:43:59.693,28,-0.878113,3.11524
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,21,-0.551875,2.16030


Now, let's represent dates in a way that would make it easier for a model to extract patterns (see chapter 4 of the attached book for more information on why we chose these particular features.)

In [5]:
# Convert our date to a pandas datetime
tabular_df["date"] = pd.to_datetime(tabular_df["CreationDate"])

# Extract meaningful features from the datetime object
tabular_df["year"] = tabular_df["date"].dt.year
tabular_df["month"] = tabular_df["date"].dt.month
tabular_df["day"] = tabular_df["date"].dt.day
tabular_df["hour"] = tabular_df["date"].dt.hour

In [6]:
tabular_df.head()

,Tags,CommentCount,CreationDate,Score,NormComment,NormScore,date,year,month,day,hour
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32,1.405553,3.66092,2010-11-18 20:40:32.857,2010,11,18,20
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,20,-0.878113,2.02388,2010-11-18 20:42:31.513,2010,11,18,20
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,34,-0.551875,3.93376,2010-11-18 20:43:28.903,2010,11,18,20
3,<plot><short-story><planning><brainstorming>,0,2010-11-18T20:43:59.693,28,-0.878113,3.11524,2010-11-18 20:43:59.693,2010,11,18,20
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,21,-0.551875,2.16030,2010-11-18 20:45:44.067,2010,11,18,20


And finally let's transform tags into dummy variables using pandas' [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) function, with each tag being assigned an index that will take the value "1" only if it is present in the given row.

In [7]:
# Select our tags, represented as strings, and transform them into arrays of tags
tags = tabular_df["Tags"]
clean_tags = tags.str.split("><").apply(
    lambda x: [a.strip("<").strip(">") for a in x])

# Use pandas' get_dummies to get dummy values 
# select only tags that appear over 500 times
tag_columns = pd.get_dummies(clean_tags.apply(pd.Series).stack()).sum(level=0)
all_tags = tag_columns.astype(bool).sum(axis=0).sort_values(ascending=False)
top_tags = all_tags[all_tags > 500]
top_tag_columns = tag_columns[top_tags.index]


In [8]:
top_tag_columns.head()

,creative-writing,fiction,style,characters,technique,novel,publishing
0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0
2,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0


In [9]:
# Add our tags back into our initial DataFrame
final = pd.concat([tabular_df, top_tag_columns], axis=1)

# Keeping only the vectorized features
col_to_keep = ["year", "month", "day", "hour", "NormComment",
               "NormScore"] + list(top_tags.index)
final_features = final[col_to_keep]

In [10]:
final_features.head()

,year,month,day,hour,NormComment,NormScore,creative-writing,fiction,style,characters,technique,novel,publishing
0,2010,11,18,20,1.405553,3.66092,0,0,0,0,0,0,0
1,2010,11,18,20,-0.878113,2.02388,0,1,0,0,0,0,0
2,2010,11,18,20,-0.551875,3.93376,0,0,0,0,0,1,1
3,2010,11,18,20,-0.878113,3.11524,0,0,0,0,0,0,0
4,2010,11,18,20,-0.551875,2.16030,0,1,0,0,0,0,0


Voila! Our tabular data is now ready to be used for a model.